In [2]:
import pandas as pd
import numpy as np
import math

In [2]:
#ratings = pd.read_csv('movielens-20m-dataset/rating.csv')

In [3]:
#len(ratings['movieId'].unique())

26744

In [20]:
#ratings = ratings[['userId', 'movieId', 'rating']]

In [30]:
#ratings[(ratings['userId'] < 10000000) && (ratings['movieId'] < )].nunique()

userId     138493
movieId     26744
rating         10
dtype: int64

In [ ]:
#movie_ids_unique = ratings['movieId'].unique()

In [6]:
#movie_ids_unique_ls = list(movie_ids_unique)

In [7]:
#def process_movie_id(old_id):
#    return movie_ids_unique_ls.index(old_id)
    

In [8]:
#list(movie_ids_unique).index(2)

0

In [9]:
#ratings['movieId'] = ratings['movieId'].apply(process_movie_id)

KeyboardInterrupt: 

In [ ]:
#ratings['movieId'].unique()

In [ ]:
#ratings.to_pickle('ratings_processed.pkl')

In [3]:
ratings = pd.read_pickle('ratings_processed.pkl')

In [4]:
ratings

,userId,movieId,rating,timestamp
0,1,0,3.5,2005-04-02 23:53:47
1,1,1,3.5,2005-04-02 23:31:16
2,1,2,3.5,2005-04-02 23:33:39
3,1,3,3.5,2005-04-02 23:32:07
4,1,4,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,1814,4.5,2009-11-13 15:42:00
20000259,138493,1037,4.5,2009-12-03 18:31:48
20000260,138493,3950,3.0,2009-12-07 18:10:57
20000261,138493,1818,5.0,2009-11-13 15:42:24


In [5]:
len(ratings)

20000263

In [6]:
ratings[['userId', 'movieId']].groupby('userId').agg(lambda x: len(list(x)))['movieId'].median()

68.0

We see that there are no ratings of $0$, all ratings are between $0.5$ and $5.$

In [7]:
ratings['rating'].unique()

array([3.5, 4. , 3. , 4.5, 5. , 2. , 1. , 2.5, 0.5, 1.5])

In [8]:
#grouped_ratings = ratings[['userId', 'movieId', 'rating']].groupby('userId').agg(list)

In [9]:
#grouped_ratings

In [10]:
#n_users = len(grouped_ratings)
#n_products =len(ratings['movieId'].unique())

In [8]:
#arr = np.zeros((n_users, n_products), dtype = np.float32)

In [9]:
#for i, row in grouped_ratings.iterrows():
#    # i is user id
#    movies_ = row['movieId']
#    ratings_ = row['rating']
#    for movie_id, rating in zip(movies_, ratings_):
#        arr[i-1, movie_id]  = rating
    

In [11]:
ratings = pd.read_pickle('df.pkl')

In [12]:
ratings['movieId'].nunique()

16872

In [13]:
from numpy.linalg import norm
from sklearn.utils import shuffle
import torch

In [14]:
class Recommender(torch.nn.Module):
    
    def __init__(self, n_U, n_P, k):
        super(Recommender, self).__init__()
        
        self.U = torch.nn.Parameter(torch.rand(n_U, k, requires_grad=True))
        self.P = torch.nn.Parameter(torch.rand(n_P, k, requires_grad=True))
        
    def forward(self, i_user, i_product):
        user_vector = self.U[i_user-1]
        product_vector = self.P[i_product]
        return user_vector @ product_vector ## inner porduct
        

In [15]:
n_U = 10000
n_P = ratings['movieId'].nunique()
k = 50

In [16]:
ratings['movieId'].nunique()

16872

In [17]:
ratings['userId'].nunique(0)

138493

In [18]:
data = shuffle(ratings[ratings['userId'] <= n_U][['userId', 'movieId', 'rating']])
data['rating'] = data['rating'] /5
data

,userId,movieId,rating
1554507,5099,72,0.1
2347945,3588,148,0.8
4123138,6160,299,0.6
17985028,5225,4227,0.6
8653468,1864,763,0.8
...,...,...,...
15073772,6881,2247,0.4
5318792,2740,396,0.8
12279789,1244,1496,0.9
7660464,6162,646,0.6


In [19]:
data['rating'].value_counts()

0.8    413609
0.6    319988
1.0    213910
0.7    157700
0.9    111326
0.4    107668
0.5     64112
0.2     50297
0.3     19684
0.1     17102
Name: rating, dtype: int64

In [20]:
model = Recommender(n_U, n_P, k)
get_loss = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-9)

In [21]:
num_epochs=30
regularization_coeff = 1

In [117]:
model(56, 430)

tensor(2.0539, grad_fn=<DotBackward>)

In [20]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 19:13:34


In [24]:
print(f'number of ratings : {len(data)}')
losses_per_epoch = []


for epoch in range(1):
    print(f'Epoch {epoch}')
    loss_acc = 0.
   
    
    for i, (i_user, i_product, rating) in enumerate(data.values):
        ## loop through all the ratings and accumulate loss ##  
        

    
        # get loss contribution of this rating
        output = model(int(i_user), int(i_product))
        rating = torch.Tensor([rating]).squeeze()

        #reg = regularization_coeff * (model.U[int(i_user-1)].mean() + model.P[int(i_product)].mean())
        loss = get_loss(output, rating) #+ reg
        loss_acc += loss
        
        loss.backward()
        optimizer.step()
        
        if i % 10000 == 0:
            print(f'i:{i}, loss_acc relative:{loss_acc/(i+1)}')

        
    print(loss_acc)
    losses_per_epoch.append(loss_acc)
    

number of ratings : 1475396
Epoch 0
i:0, loss_acc relative:0.5124370455741882
i:10000, loss_acc relative:0.8705323934555054
i:20000, loss_acc relative:0.8589697480201721
i:30000, loss_acc relative:0.856833815574646
i:40000, loss_acc relative:0.8543593883514404
i:50000, loss_acc relative:0.8597791194915771
i:60000, loss_acc relative:0.8634073138237
i:70000, loss_acc relative:0.8675939440727234
i:80000, loss_acc relative:0.8698365092277527
i:90000, loss_acc relative:0.8722259402275085
i:100000, loss_acc relative:0.8773757815361023
i:110000, loss_acc relative:0.8790355920791626
i:120000, loss_acc relative:0.8831089735031128
i:130000, loss_acc relative:0.8859332799911499
i:140000, loss_acc relative:0.889110267162323
i:150000, loss_acc relative:0.8924996256828308
i:160000, loss_acc relative:0.8954585790634155
i:170000, loss_acc relative:0.8985914587974548
i:180000, loss_acc relative:0.9007030725479126
i:190000, loss_acc relative:0.9015217423439026
i:200000, loss_acc relative:0.9041188955307

KeyboardInterrupt: 

In [38]:
model.P

Parameter containing:
tensor([[1.6664, 1.3872, 1.6137, 1.4809, 1.7738],
        [2.0701, 2.2371, 2.5429, 2.2711, 3.1417],
        [1.0108, 0.8331, 0.2705, 1.2015, 1.4081],
        ...,
        [0.0825, 0.0663, 0.6178, 0.0853, 0.0058],
        [0.7584, 0.3502, 0.6944, 0.2116, 0.5576],
        [0.2183, 0.5071, 0.5212, 0.1593, 0.9499]], requires_grad=True)

In [13]:

for i_user, i_product, rating in ratings[['userId', 'movieId', 'rating']].values:
    ## loop through all the ratings and accumulate loss ##  
    
    # get loss contribution of this rating
    output = model(int(i_user), int(i_product))
    rating = torch.Tensor([rating]).squeeze()

    loss = get_loss(output, rating) + regularization_coeff * (model.U[i_user-1].mean() + model.P[i_product].mean())
   
    
    loss.backward()
    optimizer.step()
    
    if i_user % 10000 == 0:
        print(loss)




    

IndexError: only integers, slices (`:`), ellipsis (`...`), None and long or byte Variables are valid indices (got numpy.float64)

In [41]:
loss_acc

tensor([1.2842e+08])

In [42]:
model.U.mean() + model.P.mean()

tensor(1.0013)

In [62]:
torch.Tensor([1,2,3]) @ torch.Tensor([4,5,6])

tensor(32.)

In [64]:
print(model(1,2))

tensor(3.9327, grad_fn=<SumBackward0>)


In [66]:
 print(f'out: {model(1,2)}')

out: 3.9326839447021484


In [70]:
model(1,2)

tensor(3.9327, grad_fn=<SumBackward0>)

In [72]:
torch.Tensor([1]).squeeze()

tensor(1.)